In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import yaml 
import numpy as np
import folium
from pyproj import Proj, transform
from h3 import h3
import psycopg2
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
conn.close()

In [2]:
def visualizar_tramo_folium(tramos,tramo_id):

    tramo1 = tramos.loc[tramos.tramo_id==tramo_id,:].iloc[0]

    #si no hay origen siguiente, tomo el primero del dia 
    if tramo_id == tramos.tramo_id.iloc[-1]:
        origen_siguiente = tramos.tramo_id.iloc[0]
    else:
        origen_siguiente = tramo_id+1
        
    tramo2 = tramos.loc[tramos.tramo_id==origen_siguiente,:].iloc[0]
    
    
    if tramo1.h3_d is None:
        pass
    else:
        tramo1.lat_d = h3.h3_to_geo(tramo1.h3_d)[0]
        tramo1.lon_d = h3.h3_to_geo(tramo1.h3_d)[1]
        
    linea_trx = tramo1.linea
    
    if linea_trx in equivalencias.keys():
        linea_carto = equivalencias[linea_trx]
    else:
        linea_carto = linea_trx
    
    paradas_destino_posibles = paradas.loc[paradas.LINEA == linea_carto,['LATITUDE','LONGITUDE']]
    if tramo1.h3_d is None:
        miny = min(tramo1.lat_o, tramo2.lat_o) - 0.06
        maxy = max(tramo1.lat_o, tramo2.lat_o) + 0.06
        minx = min(tramo1.lon_o, tramo2.lon_o) - 0.06
        maxx = max(tramo1.lon_o, tramo2.lon_o) + 0.06
    else:
        miny = min(tramo1.lat_o, tramo2.lat_o) - 0.06
        maxy = max(tramo1.lat_o, tramo2.lat_o) + 0.06
        minx = min(tramo1.lon_o, tramo2.lon_o) - 0.06
        maxx = max(tramo1.lon_o, tramo2.lon_o) + 0.06


    x_centro,y_centro = minx + ((maxx - minx)/2),miny + ((maxy - miny)/2)

    m = folium.Map(
        location=[y_centro,x_centro],
        zoom_start=12,
        tiles='cartodbpositron'
    )
    
    for i in paradas_destino_posibles.index:
        folium.CircleMarker(
            radius=1,
            location=[paradas_destino_posibles.LATITUDE.loc[i],paradas_destino_posibles.LONGITUDE.loc[i]],
            color='black',
            fill=True,
            fill_color='black'
        ).add_to(m)
        
        
    folium.Circle(
        radius=100,
        location=[tramo1.lat_o,tramo1.lon_o],
        popup='Origen '+str(tramo1.fecha).split(' ')[1],
        color='crimson',
        fill=False,
    ).add_to(m)
    
    if tramo1.h3_d is None:
        pass
    else:

        folium.Marker(
            location=[tramo1.lat_d,tramo1.lon_d],
            popup='Destino',
            icon=folium.Icon(icon='glyphicon-remove-circle',color='red')
        ).add_to(m)
    
    
    folium.Circle(
        radius=100,
        location=[tramo2.lat_o,tramo2.lon_o],
        popup='Origen 2 '+str(tramo2.fecha).split(' ')[1],
        color='grey',
        fill=False,
    ).add_to(m)

                  
    return m

In [3]:
DB_USERNAME = 'sube_user'
DB_PASSWORD = 'sube_pass'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sube'
DB_SCHEMA = 'public'

In [4]:
resolucion = 11

In [5]:
with open('../recorridos_equivalencias.yaml') as file:
    equivalencias = yaml.load(file, Loader=yaml.FullLoader)

In [6]:
# Conectar a la db
conn = psycopg2.connect(user = DB_USERNAME,
                                      password = DB_PASSWORD,
                                      host = DB_HOST,
                                      port = DB_PORT,
                                      database = DB_NAME)

In [7]:
q = """
select p."LINEA", p."LATITUDE",p."LONGITUDE",p."h3_res_%i"
from paradas p
"""%resolucion
paradas = pd.read_sql(q, conn)

In [8]:
# traer trx de tarjetas que hayan pasado por la B SIN MISSING
q = """
select distinct tarjeta
FROM tramos_linea_b
where h3_d is not null
"""
tarjetas_con_destino = pd.read_sql(q, conn)

In [9]:
# traer trx de tarjetas que hayan pasado por la B CON MISSING
q = """
select distinct tarjeta
FROM tramos_linea_b
where h3_d is null
"""
tarjetas_con_destino = pd.read_sql(q, conn)

In [9]:
# traer trx de tarjetas que hayan pasado por la B 
q = """
select distinct tarjeta
FROM tramos_linea_b
"""
tarjetas_con_destino = pd.read_sql(q, conn)

In [34]:
#1263039 combina tren bus y subte
#1233654 tipico pendular
#1248216 ejemplo de perder un caso
#18082 ejemplo de perder un caso porque camina mucho (actividad mediante quizas)
#9099909 ejemplo de tarjeta que no respeta el ultimo viaje con destino 
tarjeta = tarjetas_con_destino.sample(1).iloc[0,0]

In [19]:
tarjeta = 147

In [20]:
q = """
select *
FROM tramos_linea_b
where tarjeta = %i
order by tramo_id
"""%tarjeta
tramos = pd.read_sql(q, conn)
tramos.sort_values('tramo_id').iloc[:,1:]

,tramo_id,linea,fecha,lat_o,lon_o,delta,h3_o,lat_d,lon_d,h3_d,viaje_id
0,0,LINEA 175,2018-10-10 06:36:45,-34.574850,-58.493470,1.131000e+12,8bc2e310954cfff,None,None,8bc2e30242e3fff,0
1,1,LINEA D,2018-10-10 06:55:42,-34.555642,-58.462378,8.455000e+12,8bc2e30242e2fff,None,None,8bc2e3118389fff,0
2,2,LINEA D,2018-10-10 09:16:44,-34.581411,-58.421196,1.670000e+12,8bc2e3118389fff,None,None,8bc2e30242e2fff,1
3,3,LINEA 175,2018-10-10 09:44:39,-34.555680,-58.462390,5.632000e+12,8bc2e30242e2fff,None,None,8bc2e3109483fff,1
4,4,LINEA B,2018-10-10 11:18:31,-34.574319,-58.486385,4.937000e+12,8bc2e31094b1fff,None,None,8bc2e3108636fff,2
5,5,LINEA B,2018-10-10 12:40:48,-34.581249,-58.474241,NaN,8bc2e3108636fff,None,None,8bc2e31094b1fff,2


In [28]:
visualizar_tramo_folium(tramos,5)